# M1. Actividad
Alumno: Javier E. Agostini Castilla - A00827216

**Instrucciones**

Realizar una simulación en donde, a partir de variables iniciales como dimensiones de la habitación, porcentaje de celdas sucias y agentes limpiadores, con el paso del tiempo los agentes limpiadores comiencen a moverse por la habitación limpiando celdas sucias.


**Instalación de Librerias**

In [48]:
!pip3 install mesa

In [49]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid
from mesa.space import SingleGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

from mesa.space import ContinuousSpace


# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

**Declaración de Variables**

In [50]:
# Dimensiones de la habitacion
M = 8
N = 8

# Cantidad de agentes (limpiadores)
agent_qty = 1

# Porcentaje de celdas sucias (0-1)
dirty_cell_percentage = 0.5

# Tiempo maximo de ejecucion en segundos
max_exe_time = 5

**Desarrollo de Clases de Modelo y Agentes**

In [51]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Cleaner):
                grid[x][y] = 2
            else:
                grid[x][y] = content.live
    return grid

class Floor(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    if np.random.choice(100) > dirty_cell_percentage * 100:
      self.live = 0
    else:
      self.live = 1

class Cleaner(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=True,
      include_center=False)
    
    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)

  def dirty(self):
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    if this_cell[0].live == 1:
      this_cell[0].live = 0
      return True
    else:
      return False

  def step(self):
    if not self.dirty():
      self.move()
  
class Room(Model):
  def __init__(self, m, n, agent_qty, dirty_cell_percentage, max_exe_time):
    self.agent_qty = agent_qty
    self.dirty_cell_percentage = dirty_cell_percentage
    self.clean_cell_percentage = 1 - dirty_cell_percentage
    self.start_time = time.time()
    self.max_exe_time = max_exe_time
    self.exe_time = None
    self.grid = MultiGrid(m, n, True)
    self.schedule = SimultaneousActivation(self)

    for (content, x, y) in self.grid.coord_iter():
      floor = Floor((x, y), self)
      self.grid.place_agent(floor, (x, y))
      self.schedule.add(floor)
        
        
    for i in range(agent_qty):
      cleaner = Cleaner(i,self)
      self.grid.place_agent(cleaner, (0, 0))
      self.schedule.add(cleaner)

    self.datacollector = DataCollector(
      model_reporters={"Grid": get_grid},
      agent_reporters={"Move": lambda a: getattr(a, "advance", None)}
    )

  def count_cells_clean(self):
    clean_cells = 0
    for cell in self.grid.coord_iter():
      cell_content, x, y = cell
      if cell_content[0].live == 0:
        clean_cells = clean_cells + 1
        return clean_cells

    #self.clean_cell_percentage = clean_cells / (self.grid.width * self.grid.height)

    #return clean_cell_percentage

  def all_cells_clean(self):
    clean_cells = 0
    for cell in self.grid.coord_iter():
      cell_content, x, y = cell
      if cell_content[0].live == 0:
        clean_cells = clean_cells + 1
      
    self.clean_cell_percentage = clean_cells / (self.grid.width * self.grid.height)
    if self.clean_cell_percentage == 1:
      return True
    else: 
      return False

            
  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()

**Ejecución**

In [52]:
start_time = time.time()
model = Room(M, N, agent_qty, dirty_cell_percentage, max_exe_time)

moves = 0
while ((time.time() - start_time) < max_exe_time or not model.all_cells_clean()):
  model.step()
  moves += 1

exe_time = str(datetime.timedelta(seconds = (time.time() - start_time)))

**Información de Ejecución**

In [53]:
print('Tiempo de ejecución:', exe_time)
print('Movimientos realizados: ', moves)
print('Porcentaje de celdas limpias: ', model.count_cells_clean() * 100, '%')

Tiempo de ejecución: 0:00:05.000316
Movimientos realizados:  16105
Porcentaje de celdas limpias:  100 %


**Visualización**

In [54]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [55]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Reds')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=50)

In [56]:
anim

**Conclusión**

A pesar de que se me hizo compleja la actividad, considero que aprendí mucho sobre agentes y practiqué mis habilidades en Python. 

Pude observar que, al aumentar la cantidad de agentes, la cantidad de pasos necesarios para limpiar la habitación completamente, así como el tiempo de ejecución se disminuyen considerablemente. Es por eso que, simuladores como este, desarrollados en base a sistemas reales, permiten que las personas puedan experimentar y entender diversas situaciones y como ciertas acciones afectan el resultado final. 

Es decir, si se tratara de un negocio, podríamos graficar el impacto de la cantidad de agentes y los pasos o tiempo necesario para limpiar la habitación y encontrar la cantidad óptima de agentes considerando presupuesto u otros factores.